# 现实网络问题诊断

网络流量分析是一个动态过程。不同网络、不同时段的网络流量都各有不同。

通过下面典型场景的分析，我们可以牢固掌握有用的实践技能。

## 案例1：无法访问Internet

### 场景描述

典型小规模网络环境: 该用户和其他客户机、服务器都连接在一个简单交换机上，连接外网的是一个路由器（默认网关），IP地址由DHCP提供。

### 问题描述

用户不能访问互联网。

已经确认该用户可以访问内网资源，包括其它工作站的共享内容，以及运行本地服务器上的应用程序。

### 故障诊断

#### 侦听线路

为了找出问题原因，我们可以在用户尝试上网的同时抓取网络流量(监听）。嗅探器的安装位置可以参考之前的讲义。

我们抓取到的数据文件为nowebacces1.pcap。

#### 分析

上述文件中记录了两个ARP分组和7个DNS分组。

1. 分析ARP分组

第1个ARP分组，发出了ARP请求，信息摘要为Who has 172.16.0.10？ Tell 172.16.0.8

第2个ARP分组，对ARP请求进行了响应，信息摘要为172.16.0.10 is at 00:24:81:a1:f6:79

172.16.0.10可能是一个网关（路由器）IP地址。（这一点，网络管理员一般会给出判断。)

2. 分析DNS分组

可以看到分组3-9都是	DNS Standard query 0x80d1 A www.google.com

正常情况下，DNS服务器会很快响应DNS查询，但在该样本中DNS查询重发了7条，也没有收到响应。

DNS查询过程持续了8秒左右。这个时间大约是浏览器报告“超时错误”的时间。

看来，浏览器无法上网的原因，应该是DNS服务器没有响应

#### 总结

- "网关"172.16.0.10在线、正常
- DNS请求失败

假设局域网内其它用户上网没有问题，那么说明远端DNS服务器没问题。

由于这个案例中，IP地址是DHCP分配的，而不是用户静态配置的，所以问题可能出现在该主机没有正确获得DHCP分配的IP地址。

通过运行ipconfig或ifconfig命令，查看IP地址，发现本机IP地址172.16.0.8是用户自己配置的静态IP，而不是DHCP分配的，所以没有获得正确网关地址，也无法正确获得DNS服务器地址。

用户需要手动修改IPv4属性设置，改为“自动获取IP地址”。

问题解决。

**这个案例告诉我们，事先了解本地网络的基本配置情况与Wireshark配合起来，有助于快速诊断问题。**

## 案例2：无法访问Google

### 场景描述

典型小规模网络环境: 该用户和其他客户机、服务器都连接在一个简单交换机上，连接外网的是一个路由器（默认网关），IP地址由DHCP提供。

### 问题描述

用户可以访问互联网，但是不能访问Google主页。浏览器显示“该页无法显示”。

已经确认该用户可以访问内网资源，包括其它工作站的共享内容，以及运行本地服务器上的应用程序。

### 故障诊断

#### 侦听线路

为了找出问题原因，我们可以在用户尝试访问google主页的同时抓取网络流量(监听）。嗅探器的安装位置可以参考之前的讲义。

我们抓取到的数据文件为nowebacces2.pcap。

#### 分析

上述文件中记录了两个ARP分组、6个TCP分组。

1. 分析ARP分组

第1个ARP分组，发出了ARP请求，信息摘要为Who has 172.16.0.102？ Tell 172.16.0.8

第2个ARP分组，对ARP请求进行了响应，信息摘要为172.16.0.102 is at 00:21:70:c0:56:f0

172.16.0.102可能是一个网关（路由器）IP地址。（这一点，网络管理员一般会给出判断。)

2. 分析TCP分组

第3个分组，为TCP分组，摘要信息：1074 → 80 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=4 SACK_PERM=1。

说明172.16.0.8尝试与172.16.0.102建立TCP连接。

第4个分组，为TCP分组，摘要信息：80 → 1074 [RST, ACK] Seq=1 Ack=1 Win=0 Len=0。

说明172.16.0.102想结束连接。

之后的第5、第6与第7、第8都是在重复上述3、4分组的过程。

#### 总结

- "网关"172.16.0.102在线、正常
- 缺少查找www.google.com 的DNS数据分组，而是将172.16.0.102作为了web服务器（google）
- TCP连接失败，172.16.0.102的80端口不能建立TCP连接。

通过检查主机本地的C:\Windows\System32\drivers\etc\hosts文件，发现里面有错误配置信息：

> 172.16.0.102     www.google.com      

这一设置令操作系统不再进行DNS查询，直接引用这一ip地址与服务器通信。

将此设置移除后，就可以访问Google主页了。

> 现实中，这类情况并不少见，一些恶意软件会主动修改hosts文件，把用户重定向到“冒牌”网站，进行网络钓鱼。

**这个案例告诉我们，我们需要掌握网络通信的基本过程，才能发现异常过程、异常IP地址。**

#### 补充：OS域名解析过程

例如：预查寻www.baidu.com 的IP地址，操作系统通常会按如下顺序进行：

- 第一步.浏览器搜索自己的DNS缓存信息，有则直接引用；无则执行第二步。

以chrome浏览器为例，可以使用“chrome://chrome-urls/” 找到“net-internals”，然后查看DNS缓存信息。


- 第二步.搜索OS中的DNS缓存，有则引用；无则执行第三步。

可以使用命令 ipconfig /displaydns > c:/dnscache.txt 将OS缓存的信息全部导出到某个文件中进行查看。

如果要清空该表，可以使用ipconfig /flushdns刷新清空；之后重启DNSClient、DHCPClient服务。

- 第三步. 搜索OS的hosts文件，有则引用；无则执行第四步。

Windows中，这个文件存放在C:\Windows\System32\drivers\etc\hosts。

Linux中，这个文件存放在\etc\hosts。

- 第四步. OS将域名发送至本地域名服务器（运行递归查询），本地域名服务器查自己的DNS缓存，有则引用；无则执行第五步。

- 第五步. 报告无法访问网页。

## 案例3：局域网内所有人均无法访问Google

### 场景描述

典型小规模网络环境: 该用户和其他客户机、服务器都连接在一个简单交换机上，连接外网的是一个路由器（默认网关），IP地址由DHCP提供。

### 问题描述

所有局域网内用户可以访问互联网，但是不能访问Google主页。浏览器显示“该页无法显示”。

已经确认该用户可以访问内网资源，包括其它工作站的共享内容，以及运行本地服务器上的应用程序。

### 故障诊断

#### 侦听线路

为了找出问题原因，我们可以在用户尝试访问google主页的同时抓取网络流量(监听）。

需要注意的是：**全网问题，一般在客户端进行监听是无法了解问题根源的，应该是在交换机镜像口或路由器上进行监听**

我们抓取到的数据文件为nowebacces3.pcap。

#### 分析

上述文件中记录了2个DNS分组、3个TCP分组。

虽然没有之前的ARP分组，可以假设地址解析过程已经完成。

1. 分析DNS分组

第1个分组，发出了DNS标准查询,摘要如下：

> 1	0.000000	172.16.0.8	4.2.2.1	DNS	74	UDP	Standard query 0x6138 A www.google.com

172.16.0.8是本地某一客户端的IP地址，4.2.2.1是DNS服务器

第2个分组，是DNS应答信息，摘要如下：

> 2	0.010440	4.2.2.1	172.16.0.8	DNS	190	UDP	Standard query response 0x6138 A www.google.com CNAME www.l.google.com A 74.125.95.105 A 74.125.95.106 A 74.125.95.147 A 74.125.95.99 A 74.125.95.103 A 74.125.95.104

从返回结果看，Google网站存在了多个IP地址。DNS查询过程正常。

2. 分析TCP分组

第3个分组，向74.125.95.105：80发出了TCP连接请求(SYN)，摘要如下：

> 3	0.014421	172.16.0.8	74.125.95.105	TCP	66	HTTP	1251 → 80 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=4 SACK_PERM=1

若是正常的TCP连接建立过程，下一分组将是74.125.95.105 应答的TCP[SYN ACK]报文。

第4个分组，仍是向74.125.95.105：80 重发TCP连接请求(SYN)，摘要如下：

> 4	0.019417	172.16.0.8	74.125.95.105	TCP	66	Bad TCP	[TCP Retransmission] 1251 → 80 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=4 SACK_PERM=1

第5个分组，仍是向74.125.95.105：80 重发TCP连接请求(SYN)，摘要如下：

> 5	1.016531	172.16.0.8	74.125.95.105	TCP	66	Bad TCP	[TCP Retransmission] 1251 → 80 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=4 SACK_PERM=1


分组4、5的出现，说明74.125.95.105：80没有响应，172.16.0.8在规定时间（重传时限）没有收到应答，就重发TCP SYN请求。

正是由于无法与Google服务器建立TCP连接，所以在1秒后，浏览器报告“无法找到网页”。

#### 总结

- DNS系统工作正常；
- TCP连接失败，74.125.95.105：80没有响应；

从经验来看，原因可能有：

- Web服务器配置错误
- Web服务器的协议栈（服务器OS的网络服务程序）工作异常（需要重启）
- 网络部署了数据包过滤，阻止本地IP访问（防火墙）

从以上情形来看，我们能做的就是检查有无本地防火墙设置，是否阻止了访问。其它问题，有待远端服务器自己解决。

现实中，这种分析有一定意义，通过抓包分析将事实分析清楚，有利于明确责任。



## 案例4：打印机故障

### 场景描述

典型小规模网络环境: 该用户和其他客户机、服务器都连接在一个简单交换机上，连接外网的是一个路由器（默认网关），IP地址由DHCP提供。

### 问题描述

局域网内某用户报告说打印机出现故障。当用户发送打印作业时，打印机能打印几页，但之后就停止工作了。改动打印设置，没有效果。

已经确认该用户可以访问内网资源，包括其它工作站的共享内容，以及运行本地服务器上的应用程序。

### 故障诊断

#### 侦听线路

为了找出问题原因，我们可以在用户尝试给打印机发送任务的同时抓取网络流量(监听）。

最好的监听位置是配置交换机，对连接打印机的接口做镜像监听。

我们抓取到的数据文件为inconsistent_printer。

#### 分析

上述文件中记录了122个TCP分组。

虽然没有之前的ARP分组，可以假设地址解析过程已经完成。

第1-3分组，在172.16.0.8：3527与172.16.0.253：9100之间建立了TCP连接。

第4-119分组，TCP数据传输过程一直保持正常。

地址为172.16.0.8的客户机，不断发送数据（打印业务）给地址为172.16.0.253的打印机；反过来，打印机进行批量确认。

第120分组，出现异常，摘要为：

> 120	15.117035	172.16.0.8	172.16.0.253	TCP	1146	Bad TCP	[TCP Window Full] 3527 → 9100 [ACK] Seq=118261 Ack=107 Win=65592 Len=1092

再查看TCP Analysis Flags信息（TCP选项）有：

> Expert Info (Warning/Sequence): TCP window specified by the receiver is now completely full

这一分组报告“172.16.0.253的TCP窗口满了”，这说明打印机的内部缓存已经全被占用。

分组121和122是120的重传数据包，从第120-122每个分组的时间间隔大约5秒钟，分组121和122的重传超时RTO为5.5秒和11.1秒。

之后，打印机就完全停止工作了。


#### 总结

- 客户机工作正常；
- 打印机内存有故障；

不是网络问题，而是打印机自身故障。需要对打印机进行重启或Reset。

**这个案例告诉我们，分析TCP重传，有助于确定故障原因**



## 案例5：分公司之困

### 场景描述

一家公司有一个总部和几家分公司，有一台Windows域控制器服务器和一台备用域控制器，以及其它常见网络设备。

域控制器是一个代理DNS服务器，它接受来自总部的上游DNS服务器的域名记录信息。

### 问题描述

当网络部署部门在分公司安装新网络设备时，发现分公司内所有用户都不能通过互联网，访问总公司内部的Web服务器。

### 故障诊断

#### 侦听线路

为了找出问题原因，我们需要在总部和分公司的通信线路上抓取网络流量(监听）。

由于问题可能存在于：分公司客户端、分公司DNS服务器、总公司路由器等多个部位，所以需要在多个地方进行监听。

这种情况下，通常要利用交换机的镜像功能。

我们抓取到的数据文件为 stranded_client.pcap、stranded_branchdns.pcap。

#### 分析

1. 分析stranded_client样本文件。

这个文件中有两个DNS分组，其摘要分别为

> 1	0.000000	172.16.16.101	172.16.16.251	DNS	69	UDP	Standard query 0x0003 A appserver

> 2	0.000346	172.16.16.251	172.16.16.101	DNS	69	UDP	Standard query response 0x0003 Server failure A appserver

不难看出，在查询appserver这个名字时，DNS服务器报告了一个错误，说明DNS服务器有问题。

既然DNS有问题，我们就查看一下公司的DNS服务器，利用交换机将这台服务器上的流量镜像到监听口。

2. 分析stranded_branchdns样本文件。

在这个文件中，前两个数据包事实上我们已经分析过了。

第三个数据包是一个TCP连接请求：

> 3	0.000120	172.16.16.251	172.16.16.250	TCP	66	TCP SYN/FIN	49160 → 53 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=256 SACK_PERM=1

这是分公司服务器172.16.16.251在申请与总公司DNS服务器172.16.16.250建立连接，以求获取DNS信息（一般是区域传送）。

但是连接没有应答。

#### 总结

- 总公司和分公司的DNS服务器之间无法进行TCP通信

后来发现，总公司的路由器只允许UDP53端口入站，而不允许TCP 53端口入站。

## 案例6：生气的开发者

### 场景描述

在IT公司，开发人员与运维人员（网络管理）会经常吵架，在遇到应用系统故障时会互相指责。

开发人员制作了一个应用程序，跟踪多个商店的销售报表并传回中心服务器。传送方式是每天晚上自动由各个网点传送一个CSV文件给中心数据库。

### 问题描述

新开发的应用程序工作不正常。服务器接收到了各个商店传回的文件，但插入数据库的数据是错误的。

一些地区的数据丢失了，有的数据有错误，有的文件中内容部分丢失了。

运维人员很苦恼，因为程序员抱怨说“网络不稳定”，一口咬定是网管的责任。

### 故障诊断

#### 侦听线路

为了收集数据，可以在一个商店或中心机房捕获数据包。

故障影响到了所有商店，因此如果这确实是网络导致的问题，那么肯定在中心机房发现问题。

我们在交换机上设置镜像，监听某个商店上传文件到服务器时的网络流量样本。

我们抓取到的数据文件为 tickedoffdeveloper.pcap。

#### 分析

我们事先是不清楚应用程序的工作机制的，只能先从流量上进行分析。这个样本文件中有93个分组。

1. 前3个分组是在172.16.16.128：2555与172.16.16.121：21间建立TCP连接。

连接建立成功。TCP 21端口是FTP服务常用端口。

2. 从第4个分组开始，到第82个分组，都是FTP通信分组

使用FTP协议，较为适合传递文件。这与背景相符。

看一下“统计”-“流量图”，可以更加清楚FTP的通信过程。

从flow graph中可以看到有很多FTP request 和 FTP response分组。

利用FTP协议向服务器上传文件，使用的是FTP协议的STOR命令，这个过程可以在wireshark中设置如下显示过滤器来查看：

> ftp.request.command == "STOR"

过滤后，可以发现第64个数据包发送了"STOR"命令，而第65个数据包进行了确认：

> 64	4.369659	172.16.16.128	172.16.16.121	FTP	83	TCP	Request: STOR store4829-03222010.csv
> 65	4.370606	172.16.16.121	172.16.16.128	FTP	79	TCP	Response: 150 Connection accepted

第66个数据包，wireshark解析该分组的协议为FTP-DATA.

> 66	4.370815	172.16.16.128	172.16.16.121	FTP-DATA	1514	TCP	FTP Data: 1460 bytes (PASV) (STOR store4829-03222010.csv)

说明传输开始，这部分证明了程序员设计的上传文件功能是可以启动的（但不一定正常结束了）。

为了排查问题，我们下面需要提取FTP文件，查看以下文件有无被损坏。这可以使用“追踪流”-"TCP流"实现。

可以从弹出的TCP流跟踪界面中看到大量信息，这些数据以明文传送，但不清楚是否完整。这时我们可以索取原始文件进行比对。

#### 总结

- 如果与原始文件比对的结果是两者一致，那么就可以证明网络传输没有问题，问题发生在应用程序中。

- 如果与原始文件比对的结果是两者不一致，那么需要排查网络故障。

**应用程序的构成往往是复杂的，找到其中的bug不容易。但多数网络应用程序所使用的网络协议较为简单，通过分析协议数据可以轻松定位问题原因。**